In [5]:
import torch
import torch.nn as nn

class Lenet(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x

model  =torch.load('model_all.pth',weights_only=False) #model_all.pth
model

RuntimeError: PytorchStreamReader failed locating file data.pkl: file not found